# The Blender Notebook Tutorial


Every year I spend a few days researching new technology and software tools. The almost exponential growth of the IT industry is only possible through repeated breakthroughs and continuous investments in the latest technological inventions.
The world of open source software can't stop surprising me with equally impressive advances.

I spent a few days this year celebrating my passion for 3D animation by watching Blender tutorials with my kids. Since Blender is free, we had no problem experimenting ourselves. I supported them by showing the techniques and they presented me some challenges. Fair business.

Before we dive into our main topic, let me include here some links I stumbled upon this year:

- [PyTorch3D](https://pytorch3d.org/)

If you don't know what hack PyTorch3D is (as I didn't know), check out the following video:

- [Building 3D deep learning models with PyTorch3D](https://www.youtube.com/watch?v=0JEb7knenps)

Why do I get the feeling they use Blender in this video? Just a few more click and we arrive to Colaboratory:

- [Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb)

My next search was "blender jupyter notebook kernel", and there you have it:

- [Blender Notebook](https://pypi.org/project/blender-notebook/)


In this Jupiter Notebook I will present you some concept how to use a notebooks to visualize you Blender work flow. 



# Prerequsit

In this notebook we assume that the Blender kernel is already installed and configured.

For this it is important to install the Jupyter notebook with the following command:

```shell
    pip install blender_notebook
    blender_notebook install --blender-exec="c:\Program Files\Blender Foundation\Blender 2.91\blender.exe"
```

For more details on getting started with Blender notebooks:

- [cheng-chi/blender_notebooklink](https://github.com/cheng-chi/blender_notebook)

It is important that we install and start the Jupyter notebook with the same version of Python that Blender uses.

For more information on the current API and how to get started, visit the following websites:

- [Python API Documentation](https://docs.blender.org/api/current/index.html)
- [Quickstart guide](https://docs.blender.org/api/current/info_quickstart.html)


Additional python packages required to use this notebook:

```script
  python -m pip install Image
```


## How to begin

First of all we need to import some libraries to work with blender. We also create a collection to store our object. 

In [3]:
# blender related imports
import bpy
from mathutils import Vector

# jupyter related imports
from IPython.display import Image

# used to get local environment 
import pathlib
import os.path
import time
import numpy as np
import PIL.Image
import PIL.ImageColor

exchange_path = pathlib.Path().absolute()



## How to display view port in notebook

In [4]:
def RenderViewport(name, x = 720, y = 512) :
    output = os.path.join(exchange_path, name)
    bpy.data.scenes['Scene'].render.filepath = output
    bpy.data.scenes['Scene'].render.resolution_x = x
    bpy.data.scenes['Scene'].render.resolution_y = y
          
    bpy.ops.render.opengl(animation=False, render_keyed_only=False, sequencer=False, write_still=True, view_context=True)
    return Image(filename=output) 
#    return output 

def RenderOutput(name, x = 720, y = 512) : 
    output = os.path.join(exchange_path, name)
    bpy.data.scenes['Scene'].render.resolution_x = x
    bpy.data.scenes['Scene'].render.resolution_y = y
    bpy.data.scenes['Scene'].render.image_settings.file_format = 'PNG' # 'JPEG' or 'FFMPEG'
    bpy.data.scenes['Scene'].render.image_settings.color_mode = 'RGB' # 'BW' or 'RGBA'
    bpy.data.scenes['Scene'].render.image_settings.color_depth = '8' # or '16'

    bpy.data.scenes['Scene'].render.filepath = output
    bpy.ops.render.render(use_viewport = True, write_still=True)
    return Image(filename=output) 


### How to define mesh surface using numpy

In [5]:
def NumpyMeshSurface(mesh, x,y,d) :
    from collections import defaultdict

    xN = 2*x.size
    
    # point contains blender x,y,z values; coord contains mesh grid coordinates
    oo = [{'point':(x[i],y[j],dd), 'coord':(i,j)} for j,sub_d in enumerate(d) for i,dd in enumerate(sub_d)]

    ff = defaultdict(lambda: [-1,-1,-1,-1])

    for index,dat in enumerate(oo):
      item = dat['coord'][0] + xN*dat['coord'][1]
      ff[item + 0     ][2]= index
      ff[item + 1     ][3]= index
      ff[item + xN    ][1]= index
      ff[item + xN + 1][0]= index

    faces = list(filter(lambda f: len(list(filter(lambda x: not x == -1, f)))==4, ff.values()))
    vertices = [ dat['point'] for dat in oo ]
    
    mesh.from_pydata(vertices, [], faces)


## How to reset Blender main file

We use the following command to reset the Blende main file to default:

There is some issue with read_homefile: [link](https://blender.stackexchange.com/q/51494)

In [6]:
#bpy.ops.wm.read_homefile()



## How to create a new collection

In [7]:
collection = bpy.data.collections.new(name='JupyterCollection')

# to place in root container
#bpy.context.scene.collection.children.link(collection)
# OR
# to place in a specific collection:
bpy.data.collections['Collection'].children.link(collection)

RenderViewport('CreateCollection.png')


## How to add a new camera

In [8]:
camera = bpy.data.cameras.new(name='JupyterCamera')
camera_object = bpy.data.objects.new('JupyterCamera',camera)

# to place in root container
#bpy.context.scene.collection.objects.link(camera_object)
# OR
# to place in a specific collection:
bpy.data.collections['JupyterCollection'].objects.link(camera_object)

# position to a secific location:
bpy.data.objects['JupyterCamera'].location = [15,10,3]

camera_constraint = bpy.data.objects['JupyterCamera'].constraints.new(type='TRACK_TO')
camera_constraint.name = 'Track To Cube'
camera_constraint.target = bpy.data.objects['Cube']
#camera_constraint.subtarget = 'Group'

# select the new camera as active render target
bpy.context.scene.camera = camera_object

RenderViewport('CreateCamera.png')


## How to change the position of the Cube

In [9]:
bpy.data.objects['Cube'].location = [ -5,-5,1 ]
bpy.data.objects['Cube'].scale = [ 1,1,1 ]

print(bpy.data.objects['Cube'].location)

RenderViewport('CubeMove.png')


## How to select faces of the default Cube

- Sellecting objects

The example below is based on the following post and articles:

- [Selecting faces in Python](https://blender.stackexchange.com/a/75519/113375)
- [Switch to vertex, edge, face mode in edit mode via python](https://blender.stackexchange.com/a/7069/113375)


In [10]:
def NormalInDirection( normal, direction, limit = 0.5 ):
    return direction.dot( normal ) > limit

def GoingUp( normal, limit = 0.5 ):
    return NormalInDirection( normal, Vector( (0, 0, 1 ) ), limit )

def GoingDown( normal, limit = 0.5 ):
    return NormalInDirection( normal, Vector( (0, 0, -1 ) ), limit )

def GoingSide( normal, limit = 0.5 ):
    return not ( GoingUp( normal, limit ) or GoingDown( normal, limit ) )


cube = bpy.data.objects['Cube']
prevMode = cube.mode
selectMode = bpy.context.tool_settings.mesh_select_mode

bpy.ops.object.mode_set(mode='OBJECT', toggle=False)
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_pattern(pattern = cube.name)

# Assign a tuple of 3 booleans to set Vertex, Edge, Face selection.
bpy.context.tool_settings.mesh_select_mode = (False, False, True) 

# selects faces going side
for face in cube.data.polygons:
    face.select = GoingSide( face.normal )

bpy.ops.object.mode_set(mode=prevMode, toggle=False)
bpy.context.tool_settings.mesh_select_mode = selectMode

bpy.ops.object.mode_set(mode='EDIT', toggle=False)

RenderViewport('CubeSelectFaces.png')

#bpy.ops.object.mode_set(mode='OBJECT', toggle=False)


In [11]:
bpy.ops.object.mode_set(mode='OBJECT', toggle=False)


## How to render camera to image

In [12]:
# display the render engine:
print(bpy.data.scenes['Scene'].render.engine)

#output = os.path.join(exchange_path, 'JupyterOutput.png')
#bpy.data.scenes['Scene'].render.resolution_x = 720
#bpy.data.scenes['Scene'].render.resolution_y = 512
#bpy.data.scenes['Scene'].render.image_settings.file_format = 'PNG' # 'JPEG' or 'FFMPEG'
#bpy.data.scenes['Scene'].render.image_settings.color_mode = 'RGB' # 'BW' or 'RGBA'
#bpy.data.scenes['Scene'].render.image_settings.color_depth = '8' # or '16'

#bpy.data.scenes['Scene'].render.filepath = output
#bpy.ops.render.render(use_viewport = True, write_still=True)
#Image(filename=output) 

RenderOutput('JupyterOutput.png')


In [13]:
RenderViewport('JupyterViewport.png')


## How to create new mesh

In [14]:
mesh = bpy.data.meshes.new(name='GravityMesh')
mesh_object = bpy.data.objects.new('GravityMesh',mesh)

# to place in a specific collection:
bpy.data.collections['JupyterCollection'].objects.link(mesh_object)

#mesh.from_pydata([(0,0,0),(0,1,0),(0,1,1),(0,0,1)], [], [[0,1,2,3]])

xN = 100
yN = 100
x = np.linspace(-10, 10, xN)
y = np.linspace(-10, 10, yN)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)
#d = - 10.0 / (np.sqrt(np.power(np.abs(xx),2.0) + np.power(np.abs(yy),2.0)) + 0.001) + 2.0 # + 0.5*np.sin(xx)*np.cos(yy)
idx = (d < -5)
d[idx] = -5
NumpyMeshSurface(mesh, x,y,d)

RenderViewport('JupyterNewMesh.png')


## How to create basic color texture

[color palete source](https://kinsta.com/wp-content/uploads/2020/07/colormind-pallete-generator.png)


In [15]:
im= PIL.Image.new('RGB', (1, 5))
im.putdata([
    PIL.ImageColor.getrgb("#efbc75"),
    PIL.ImageColor.getrgb("#c1e1a7"), 
    PIL.ImageColor.getrgb("#148dd8"), 
    PIL.ImageColor.getrgb("#1a4a5a"), 
    PIL.ImageColor.getrgb("#0e2c40")
])
output = os.path.join(exchange_path, 'BasicColors.png')
im.save(output)


## How to create new material with texture using nodes

In [16]:
mat = bpy.data.materials.new('Levels')
# Assign it to object
## if mesh.materials: mesh.materials[0] = mat else:
mesh.materials.append(mat)

mat.blend_method = 'BLEND'
mat.show_transparent_back = False
    
mat.use_nodes = True

bsdf = mat.node_tree.nodes["Principled BSDF"]
texImage = mat.node_tree.nodes.new('ShaderNodeTexImage')
texImage.interpolation = 'Closest'  # 'Linear', 'Cubic', 'Smart'

output = os.path.join(exchange_path, 'BasicColors.png')
img = bpy.data.images.load(output)       # load image
texImage.image = img

bsdf.inputs['Roughness'].default_value = 0.40
bsdf.inputs['Metallic'].default_value = 0.9
mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])

# OR
#bsdf.inputs['Emission Strength'].default_value = 1.5
#bsdf.inputs['Roughness'].default_value = 1.0
#mat.node_tree.links.new(bsdf.inputs['Base Color'], texImage.outputs['Color'])
#mat.node_tree.links.new(bsdf.inputs['Emission'], texImage.outputs['Color'])
#bpy.context.scene.eevee.use_bloom = True
#bpy.data.worlds["World"].node_tree.nodes["Background"].inputs['Strength'].default_value = 0.0

#bsdf.inputs['Alpha'].default_value = 0.8


## How to manipulate 3d view and project uv map 

In [17]:
# print(bpy.context.active_object)
# print(bpy.context.selected_objects)

# clear all selection
bpy.ops.object.mode_set(mode='OBJECT', toggle=False)
bpy.ops.object.select_all(action='DESELECT')

# select our new mesh
obj = bpy.data.objects['GravityMesh']
obj.select_set(True)
bpy.context.view_layer.objects.active = obj

# change shader
#bpy.ops.object.shade_flat()
bpy.ops.object.shade_smooth()

script_screen = bpy.data.screens['Layout']
#script_screen = bpy.data.screens['Scripting']
script_area = list(filter(lambda x: x.type == 'VIEW_3D', script_screen.areas))[0];
script_view3d = script_area.spaces[0]
script_region = list(filter(lambda x: x.type == 'WINDOW', script_area.regions))[0]

# change view port settings
script_view3d.shading.type = 'MATERIAL' #, 'SOLID', 'RENDERED', 'WIREFRAME'
script_view3d.shading.light = 'MATCAP' # 'STUDIO', 'MATCAP', 'FLAT'
#script_view3d.region_3d.view_perspective = 'PERSP' # 'ORTHO'


# change from object mode to edit mode
bpy.ops.object.mode_set(mode='EDIT', toggle=False)

# assign a tuple of 3 booleans to set Vertex, Edge, Face selection.
bpy.context.tool_settings.mesh_select_mode = (False, False, True) 

# select all faces to project uv map on them
bpy.ops.mesh.select_all(action='SELECT')

# prepare context override
override = bpy.context.copy()
override['area'] = script_area
override['region'] = script_region

# store current view rotation
orig = script_view3d.region_3d.view_rotation.copy()

# configure fron view
bpy.ops.view3d.view_axis(override, type='FRONT')
script_view3d.region_3d.update()

# project uv map from view 
bpy.ops.uv.project_from_view(override, camera_bounds=False, correct_aspect=True, scale_to_bounds=True)

bpy.ops.object.mode_set(mode='OBJECT', toggle=False)

# restore current view rotation
script_view3d.region_3d.view_rotation = orig
script_view3d.region_3d.view_perspective = 'PERSP' # 'ORTHO'
script_view3d.region_3d.update()

RenderViewport('JupyterConture.png')


In [18]:
RenderOutput('JupyterConturOutput.png')


## How to load and save main file

In [19]:
output = os.path.join(exchange_path, 'BlenderNotebook.blend')

bpy.ops.wm.save_mainfile(filepath=output)

# bpy.ops.wm.read_homefile(filepath=output)


## How to include video 

In [23]:
#from IPython.display import HTML

#HTML("""
#    <video alt="test" controls>
#        <source src="render_010001-0250.mkv" type="video/mp4">
#    </video>
#""")

## TODO:
- Investigate posiblity to add a viewport in the browser session.

## Additional source used to create this notebook

- [How do I get the full path of the current file's directory?](https://stackoverflow.com/a/3430395/5770014)

- [Getting started with Anaconda](https://www.anaconda.com/products/individual)
